In [1]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Lambda
import keras.applications.mobilenet_v2 as mobilenetv2
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [2]:
# Load MobileNetV2 base model without the top layer
mobilenetv2_layer = mobilenetv2.MobileNetV2(include_top=False, 
                                            input_shape=(224, 224, 3),
                                            weights='../mobilenet-v2-keras-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5')
mobilenetv2_layer.trainable = False

In [3]:
# Create the model architecture
model = Sequential()
# Input layer
model.add(tf.keras.Input(shape=(224, 224, 3)))

# Lambda preprocessing layer
def mobilenetv2_preprocessing(img):
    return mobilenetv2.preprocess_input(img)

model.add(Lambda(mobilenetv2_preprocessing))

model.add(mobilenetv2_layer)

# Add the GlobalAveragePooling2D layer
model.add(GlobalAveragePooling2D())

# Update the Dense layer for 2 classes
model.add(Dense(2, activation='softmax'))

# Compile the model with a lower learning rate for fine-tuning
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['categorical_accuracy'])

# Print the updated model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [5]:
train_dir = 'trainingData/'

In [6]:
# Create an ImageDataGenerator with data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize pixel values between 0 and 1
    shear_range=0.2,             # Randomly apply shear transformations
    zoom_range=0.2,              # Randomly zoom into images
    horizontal_flip=True,        # Randomly flip images horizontally
    validation_split=0.2         # Split 20% of the data for validation
)

# Load training data with flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),      # Resizes images to 224x224
    batch_size=32,               # Number of images per batch
    class_mode='categorical',    # Since we have two classes: metal and plastic
    subset='training'            # Use the 'training' subset
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'          # Use the 'validation' subset
)

# Print class indices to verify the labels
print(f"Class Indices: {train_generator.class_indices}")

Found 288 images belonging to 2 classes.
Found 72 images belonging to 2 classes.
Class Indices: {'metal': 0, 'plastic': 1}


In [7]:
# Fine-tune the model on the new data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
9/9 [==============================] - 7s 602ms/step - loss: 0.7265 - categorical_accuracy: 0.5000 - val_loss: 0.6942 - val_categorical_accuracy: 0.5312
Epoch 2/10
9/9 [==============================] - 4s 476ms/step - loss: 0.6989 - categorical_accuracy: 0.4757 - val_loss: 0.6997 - val_categorical_accuracy: 0.4531
Epoch 3/10
9/9 [==============================] - 4s 472ms/step - loss: 0.6939 - categorical_accuracy: 0.5000 - val_loss: 0.6968 - val_categorical_accuracy: 0.4688
Epoch 4/10
9/9 [==============================] - 4s 468ms/step - loss: 0.6945 - categorical_accuracy: 0.4965 - val_loss: 0.6899 - val_categorical_accuracy: 0.5156
Epoch 5/10
9/9 [==============================] - 4s 467ms/step - loss: 0.6921 - categorical_accuracy: 0.5312 - val_loss: 0.6923 - val_categorical_accuracy: 0.5156
Epoch 6/10
9/9 [==============================] - 4s 467ms/step - loss: 0.6909 - categorical_accuracy: 0.5417 - val_loss: 0.6920 - val_categorical_accuracy: 0.5000
Epoch 7/10
9/9 [

In [8]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# Save the fine-tuned model
model.save('fine_tuned_plastic_metal_model.h5')

3/3 [==============================] - 1s 227ms/step - loss: 0.6895 - categorical_accuracy: 0.5833
Validation Loss: 0.6894851326942444, Validation Accuracy: 0.5833333134651184


### Inference

In [9]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the fine-tuned model
model = load_model('fine_tuned_plastic_metal_model.h5')

In [10]:
img_path = '../sample_data/plastic_bottle_out_of_shape.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = mobilenetv2.preprocess_input(img_array)

In [11]:
# Predict using the fine-tuned model
predictions = model.predict(img_array)
class_index = np.argmax(predictions, axis=1)

# Map the predicted index to class label
class_labels = ['plastic', 'metal']
predicted_class = class_labels[class_index[0]]
print(f'Predicted Class: {predicted_class}')

1/1 [==============================] - 1s 532ms/step
Predicted Class: plastic
